<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/retrievers/ensemble_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble Query Engine Guide

Oftentimes when building a RAG applications there are many retreival parameters/strategies to decide from (from chunk size to vector vs. keyword vs. hybrid search, for instance).

Thought: what if we could try a bunch of strategies at once, and have any AI/reranker/LLM prune the results?

This achieves two purposes:
- Better (albeit more costly) retrieved results by pooling results from multiple strategies, assuming the reranker is good
- A way to benchmark different retrieval strategies against each other (w.r.t reranker)

This guide showcases this over the Great Gatsby. We do ensemble retrieval over different chunk sizes and also different indices.

**NOTE**: A closely related guide is our [Ensemble Retrievers Guide](https://gpt-index.readthedocs.io/en/stable/examples/retrievers/ensemble_retrieval.html) - make sure to check it out! 

## Setup

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index

In [18]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    SimpleKeywordTableIndex,
)
from llama_index.response.notebook_utils import display_response
from llama_index.llms import OpenAI

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


## Download Data

In [ ]:
!wget 'https://github.com/jerryjliu/llama_index/blob/main/examples/gatsby/gatsby_full.txt' 'gatsby_full.txt'

## Load Data

We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.

In [3]:
# try loading great gatsby

documents = SimpleDirectoryReader(
    input_files=["./gatsby_full.txt"]
).load_data()

In [4]:
# initialize service context (set chunk size)
llm = OpenAI(model="gpt-4")
chunk_sizes = [128, 256, 512, 1024]
service_contexts = []
nodes_list = []
vector_indices = []
query_engines = []
for chunk_size in chunk_sizes:
    print(f"Chunk Size: {chunk_size}")
    service_context = ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)
    service_contexts.append(service_context)
    nodes = service_context.node_parser.get_nodes_from_documents(documents)

    # add chunk size to nodes to track later
    for node in nodes:
        node.metadata["chunk_size"] = chunk_size
        node.excluded_embed_metadata_keys = ["chunk_size"]
        node.excluded_llm_metadata_keys = ["chunk_size"]

    nodes_list.append(nodes)

    # build vector index
    vector_index = VectorStoreIndex(nodes)
    vector_indices.append(vector_index)

    # query engines
    query_engines.append(vector_index.as_query_engine())

Chunk Size: 128
Chunk Size: 256
Chunk Size: 512
Chunk Size: 1024


In [5]:
# try ensemble retrieval

from llama_index.tools import RetrieverTool

retriever_tools = []
for chunk_size, vector_index in zip(chunk_sizes, vector_indices):
    retriever_tool = RetrieverTool.from_defaults(
        retriever=vector_index.as_retriever(),
        description=f"Retrieves relevant context from the Great Gatsby (chunk size {chunk_size})",
    )
    retriever_tools.append(retriever_tool)

In [6]:
from llama_index.selectors.pydantic_selectors import PydanticMultiSelector
from llama_index.retrievers import RouterRetriever


retriever = RouterRetriever(
    selector=PydanticMultiSelector.from_defaults(llm=llm, max_outputs=4),
    retriever_tools=retriever_tools,
)

In [7]:
nodes = await retriever.aretrieve(
    "Describe and summarize the interactions between Gatsby and Daisy"
)

Selecting retriever 0: This choice retrieves a moderate amount of context from the Great Gatsby, which could provide a balanced amount of detail for describing and summarizing the interactions between Gatsby and Daisy..
Selecting retriever 1: This choice retrieves a larger amount of context from the Great Gatsby, which could provide more detail for describing and summarizing the interactions between Gatsby and Daisy..
Selecting retriever 2: This choice retrieves an even larger amount of context from the Great Gatsby, which could provide a comprehensive summary of the interactions between Gatsby and Daisy..
Selecting retriever 3: This choice retrieves the largest amount of context from the Great Gatsby, which could provide the most detailed and comprehensive summary of the interactions between Gatsby and Daisy..
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=40 request_id=d269f8a582ac9a70cdb6f587a34d5877 response_code=200
message='OpenAI API respon

In [8]:
for node in nodes:
    print(node.node.metadata["chunk_size"])
    print(node.node.get_text())

128
the beach that morning. Finally we came to Gatsby’s own
apartment, a bedroom and a bath, and an Adam’s study, where we sat
down and drank a glass of some Chartreuse he took from a cupboard in
the wall.

He hadn’t once ceased looking at Daisy, and I think he revalued
everything in his house according to the measure of response it drew
from her well-loved eyes. Sometimes too, he stared around at his
possessions in a dazed
128
turn out as he had
imagined. He had intended, probably, to take what he could and go—but
now he found that he had committed himself to the following of a
grail. He knew that Daisy was extraordinary, but he didn’t realize
just how extraordinary a “nice” girl could be. She vanished into her
rich house, into her rich, full life, leaving Gatsby—nothing. He felt
married to her, that was all.

When they met again, two days later, it
256
the
direction. In this heat every extra gesture was an affront to the
common store of life.

The room, shadowed well with awnings, wa

In [9]:
# define reranker
from llama_index.indices.postprocessor import (
    LLMRerank,
    SentenceTransformerRerank,
    CohereRerank,
)

# reranker = LLMRerank()
# reranker = SentenceTransformerRerank(top_n=10)
reranker = CohereRerank(top_n=10)

In [10]:
# define RetrieverQueryEngine
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever, node_postprocessors=[reranker])

In [11]:
response = query_engine.query(
    "Describe and summarize the interactions between Gatsby and Daisy"
)

Selecting retriever 0: This choice provides a moderate chunk size that could contain relevant interactions between Gatsby and Daisy without being too overwhelming..
Selecting retriever 1: This choice provides a larger chunk size that could contain more detailed interactions between Gatsby and Daisy..
Selecting retriever 2: This choice provides an even larger chunk size that could contain extensive interactions between Gatsby and Daisy, providing a more comprehensive summary..
Selecting retriever 3: This choice provides the largest chunk size that could contain the most detailed and comprehensive interactions between Gatsby and Daisy, but it might also include a lot of irrelevant information..


In [ ]:
display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

In [13]:
# compute the average precision for each chunk size based on positioning in combined ranking
from collections import defaultdict
import pandas as pd


def mrr_all(metadata_values, metadata_key, source_nodes):
    # source nodes is a ranked list
    # go through each value, find out positioning in source_nodes
    value_to_mrr_dict = {}
    for metadata_value in metadata_values:
        mrr = 0
        for idx, source_node in enumerate(source_nodes):
            if source_node.node.metadata[metadata_key] == metadata_value:
                mrr = 1 / (idx + 1)
                break
            else:
                continue

        # normalize AP, set in dict
        value_to_mrr_dict[metadata_value] = mrr

    df = pd.DataFrame(value_to_mrr_dict, index=["MRR"])
    df.style.set_caption("Mean Reciprocal Rank")
    return df

In [14]:
# Compute the Mean Reciprocal Rank for each chunk size (higher is better)
# we can see that chunk size of 256 has the highest ranked results.
print("Mean Reciprocal Rank for each Chunk Size")
mrr_all(chunk_sizes, "chunk_size", response.source_nodes)

Mean Reciprocal Rank for each Chunk Size


,128,256,512,1024
MRR,0.2,0.166667,0.5,1.0


## Compare Against Baseline

Compare against a baseline of chunk size 1024 (k=2)

In [15]:
query_engine_1024 = query_engines[-1]

In [16]:
response_1024 = query_engine_1024.query(
    "Describe and summarize the interactions between Gatsby and Daisy"
)

In [ ]:
display_response(response_1024, show_source=True, source_length=500)